# MACHINE LEARNING II – TР 2

Objectif:

L'objectif de ce TP est de mettre en pratique les concepts fondamentaux de l'apprentissage parrenforcement en explorant l'algorithmeQ-Learning. À travers une série d'exercices progressifs.
les étudiants vont apprendre à implémenter cet algorithme, comprendre l'impact des stratégies d'exploration et d'exploitation, et analyser la convergence des valeurs Q. L'environnement FrozenLak de OpenAl Gym servira de terrain d'expérimentation, permettant d'illustrer concrètement comment un agent apprend à optimiser ses décisions grâce aux mises à jour successives de sa Q-table.

## Exercice 1: Exploration de l'environnement FrozenLake

Instructions : 
1. charger l'envirenement FrozenLake-v1 de OpenAI Gym
2. Afficher les informations de l'espace d'etats et d'actions.
3. Execute une boucle où l'agent prend des actions aleatoires pendant plusieurs episodes.
4. Observer les obeservations et les recompenses obtenus.

In [7]:
import gymnasium as gym
import numpy as np

# Charger l’environnement FrozenLake
env = gym.make("FrozenLake-v1", is_slippery=True)

# Afficher l’espace d’états et d’actions
print(" Espace d’états :", env.observation_space)
print(" Espace d’actions :", env.action_space)

num_episodes = 10  # Tu peux augmenter si tu veux

for episode in range(num_episodes):
    print(f"\n Épisode {episode + 1}")
    observation, info = env.reset()
    done = False
    step = 0

    while not done:
        action = env.action_space.sample()  # Action aléatoire
        new_observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        step += 1

        print(f"Étape {step}: état={observation}, action={action} -> nouvel état={new_observation}, récompense={reward}")
        observation = new_observation

env.close()


 Espace d’états : Discrete(16)
 Espace d’actions : Discrete(4)

 Épisode 1
Étape 1: état=0, action=2 -> nouvel état=4, récompense=0.0
Étape 2: état=4, action=2 -> nouvel état=8, récompense=0.0
Étape 3: état=8, action=0 -> nouvel état=4, récompense=0.0
Étape 4: état=4, action=0 -> nouvel état=4, récompense=0.0
Étape 5: état=4, action=0 -> nouvel état=4, récompense=0.0
Étape 6: état=4, action=3 -> nouvel état=5, récompense=0.0

 Épisode 2
Étape 1: état=0, action=2 -> nouvel état=4, récompense=0.0
Étape 2: état=4, action=0 -> nouvel état=4, récompense=0.0
Étape 3: état=4, action=2 -> nouvel état=8, récompense=0.0
Étape 4: état=8, action=1 -> nouvel état=12, récompense=0.0

 Épisode 3
Étape 1: état=0, action=2 -> nouvel état=0, récompense=0.0
Étape 2: état=0, action=1 -> nouvel état=1, récompense=0.0
Étape 3: état=1, action=1 -> nouvel état=5, récompense=0.0

 Épisode 4
Étape 1: état=0, action=3 -> nouvel état=1, récompense=0.0
Étape 2: état=1, action=0 -> nouvel état=5, récompense=0.0

 É

## Exercice 2: Implémenintion de la Q-Table et Initialisation

Instructions:
1. Créer une Q-Table de dimension (nombre d'états x nombre d'actions),
initialisée a 0.
2. Afficher la Q-Table avant l'apprentissage.
3. Vérifier que chaque état a une liste de valeurs associées aux actions possibles.

In [33]:
# Charger l’environnement
env = gym.make("FrozenLake-v1", is_slippery=True)

# Récupérer le nombre d'états et d'actions
num_states = env.observation_space.n
num_actions = env.action_space.n

# Initialisation de la Q-table à 0
q_table = np.zeros((num_states, num_actions))

# Affichage des dimensions
print(f"Nombre d’états : {num_states}")
print(f"Nombre d’actions : {num_actions}")

# Affichage de la Q-table initiale
print("\n Q-Table initialisée")
print("\n Vérification des valeurs Q par état :")
for state in range(num_states):
    print(f"État {state} → Valeurs Q : {q_table[state]}")



Nombre d’états : 16
Nombre d’actions : 4

 Q-Table initialisée

 Vérification des valeurs Q par état :
État 0 → Valeurs Q : [0. 0. 0. 0.]
État 1 → Valeurs Q : [0. 0. 0. 0.]
État 2 → Valeurs Q : [0. 0. 0. 0.]
État 3 → Valeurs Q : [0. 0. 0. 0.]
État 4 → Valeurs Q : [0. 0. 0. 0.]
État 5 → Valeurs Q : [0. 0. 0. 0.]
État 6 → Valeurs Q : [0. 0. 0. 0.]
État 7 → Valeurs Q : [0. 0. 0. 0.]
État 8 → Valeurs Q : [0. 0. 0. 0.]
État 9 → Valeurs Q : [0. 0. 0. 0.]
État 10 → Valeurs Q : [0. 0. 0. 0.]
État 11 → Valeurs Q : [0. 0. 0. 0.]
État 12 → Valeurs Q : [0. 0. 0. 0.]
État 13 → Valeurs Q : [0. 0. 0. 0.]
État 14 → Valeurs Q : [0. 0. 0. 0.]
État 15 → Valeurs Q : [0. 0. 0. 0.]


## Exercice 3: Implémentation du Q-Learning avec Mise à Jour

Instructions :

1. Définir les hyperparamètres: taux d'apprentissage (alpha), facteur de discount (gamma), epsilon pour lexploration.
2. Mettre à jour la Q-Table en appliquant la règle de mise à jour du Q-Learning:
Q(3, a) = Q(s, a) +a [R+ ymax Q(e', 2') -Q(s, a)]
3. Exécuter plusieurs épisodes et observer l'évolution de la table.

In [37]:
# Création de l’environnement
env = gym.make("FrozenLake-v1", is_slippery=True)
num_states = env.observation_space.n
num_actions = env.action_space.n

# Q-table initialisée à 0
q_table = np.zeros((num_states, num_actions))

# Hyperparamètres
alpha = 0.1            # Taux d’apprentissage
gamma = 0.99           # Facteur de discount
epsilon = 1.0          # Niveau d’exploration initial
epsilon_decay = 0.995  # Décroissance d’epsilon
min_epsilon = 0.01
num_episodes = 5000    # Nombre d’épisodes d’apprentissage

# Apprentissage
for episode in range(num_episodes):
    state, info = env.reset()
    done = False

    while not done:
        # Choisir action (ε-greedy)
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Action aléatoire
        else:
            action = np.argmax(q_table[state])  # Action selon la Q-table

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Mise à jour de la Q-table (formule Q-learning)
        best_next_action = np.max(q_table[next_state])
        q_table[state, action] = q_table[state, action] + alpha * (
            reward + gamma * best_next_action - q_table[state, action]
        )

        state = next_state

    # Mise à jour de epsilon (exploration diminue)
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

# Affichage de la Q-table finale
print("\n Q-Table après apprentissage :")
print(q_table)


 Q-Table après apprentissage :
[[0.49680779 0.45976383 0.43714378 0.4490385 ]
 [0.08699297 0.02049842 0.0212858  0.32892152]
 [0.07102829 0.07773071 0.11881115 0.17172254]
 [0.01311517 0.04665465 0.02827669 0.02697972]
 [0.50728623 0.37233412 0.33899513 0.35983718]
 [0.         0.         0.         0.        ]
 [0.20699255 0.04985315 0.07987513 0.03405345]
 [0.         0.         0.         0.        ]
 [0.35007753 0.40365046 0.49283163 0.54538629]
 [0.37295882 0.59216091 0.49351665 0.42129451]
 [0.49553654 0.32005325 0.26203592 0.18399582]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.47862796 0.49751853 0.72498161 0.34139484]
 [0.61093637 0.79019074 0.76025221 0.63718633]
 [0.         0.         0.         0.        ]]


# Exercices 4: Évaluation des Performances de l'Agent

Iustructions:


1. Lancer 100 épisodes en utilisant uniquement l'action optimale (argmax(O[s, а])). 
2. Mesurer le taux de réussite de l'agent (nombre de fois où il atteint l'objectii). 
3. Comparer les performances avec celles obtenues dans l'Exercice ! (actions aléatoires).

In [28]:
num_test_episodes = 100
successes = 0

for episode in range(num_test_episodes):
    state, info = env.reset()
    done = False

    while not done:
        # Action optimale selon la Q-table
        action = np.argmax(q_table[state])
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        state = next_state

        if done and reward == 1.0:
            successes += 1

# Taux de réussite
success_rate = (successes / num_test_episodes) * 100
print(f"\n✅ L’agent a réussi à atteindre l’objectif {successes} fois sur {num_test_episodes}.")
print(f"🎯 Taux de réussite : {success_rate:.2f} %")



✅ L’agent a réussi à atteindre l’objectif 69 fois sur 100.
🎯 Taux de réussite : 69.00 %
